# Data Understanding (hiểu dữ liệu)

### Dự đoán khả năng trả của khoản vay

### Xác định target variable (=dependent variable)

Cột **loan_status** sẽ là target variable của ta vì ta muốn dự đoán biến này. Nhìn sơ qua excel thì có thể thấy là biến này có các giá trị Fully Paid (nợ được trả), và Charged Off (nợ bị mất). Ta sẽ dùng pandas để xem biến này còn có giá trị nào khác sau.

File gốc tải về được giải nén 'LoanStats3a.csv' và lưu trong thư mục data. Kích thước file 42MB dư xăng để lưu trong bộ nhớ nên mình sẽ làm một số bước data cleaning ban đầu trong excel cho nhanh.

1. Xóa dong đầu đi, thông tin không liên quan
2. Xóa 2 dòng cuối ở row 42543 và 42544 chứa thông tin chú thích (Total amount funded in policy code 1: 460296150 va Total amount funded in policy code 2: 0)
3. Liếc sơ qua thì có 2 cột đầu id, member_id và các cột phía sau không có dữ liệu, mình sẽ dùng pandas để xóa các cột không có dữ liệu đi.

**Xóa cột không có dữ liệu**

In [1]:
import pandas as pd

In [16]:
rawfile = 'data\loans.csv'

In [21]:
%%capture 
#ignore warning output

raw = pd.read_csv(rawfile)

In [22]:
raw.head(3)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Columns: 145 entries, id to settlement_term
dtypes: float64(115), object(30)
memory usage: 47.1+ MB


có 145 columns cả thảy

In [10]:
cols = raw.columns.values #lấy headers của các columns

In [20]:
#list ra những cols co unique value = 1 (khong tinh toi missing values)

raw['id'].value_counts()

Loans that do not meet the credit policy    1
Name: id, dtype: int64

In [14]:
raw['member_id'].unique()

array([nan])

In [40]:
#dataframe.nunique() function sẽ return count của unique value của mỗi cột.
unique_count = raw.nunique()[raw.nunique()<=1]
print(unique_count[:5])
print(len(unique_count)) #ta sẽ bỏ được 92 columns

id                     1
member_id              0
pymnt_plan             1
url                    0
initial_list_status    1
dtype: int64
92


In [42]:
useless_cols = unique_count.keys() #lấy row name(index) của Series
raw = raw.drop(useless_cols, axis=1) #axis =1 -> drop các cols có tên useless_cols

In [44]:
raw.head(3)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,delinq_amnt,pub_rec_bankruptcies,tax_liens,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,...,0.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,...,0.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,...,0.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN


**145 cột ban đầu, bỏ 92 cột không có giá trị, còn 53 cột**

In [45]:
#lưu file data đang làm thành 1 file csv mới. mình dùng Excel để nhìn
#toàn bộ phần data còn lại cho nhanh
raw.to_csv('data\loan1.csv')

#### Xem xét các cột còn lại. Các cột bỏ:

1. emp_title: employer của người muốn vay. Theo mình, đây là feature chưa xác định được tầm quan trọng, ý kiến chủ quan của mình là ít. Employer rất rời rạc nên nếu chuyển về dạng số sẽ gây ra nhiễu lớn. Nên mình sẽ bỏ.

2. desc: dạng text, người muốn vay miêu tả về khoản yêu cầu được cho vay của minh (lý do, hoàn cảnh,...) ->